In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import lightgbm as lgb
import logging
import pickle
import gc

In [ ]:
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [ ]:
base_path = './data'
feature_path = './feature'
newfeature_path = './feature_test_2_ori'

In [ ]:
test = pd.read_csv(f'{base_path}/invite_info_evaluate_2_0926.txt', sep='\t', header=None)
test.columns = ['qid', 'uid', 'dt']
sub = test.copy()
sub_size = len(sub)

del test['dt']
logging.info("test %s", test.shape)

In [ ]:
# qid feature
t1 = pd.read_pickle(f'{newfeature_path}/test2_qid_feature.pkl')
t1 = t1.reset_index(drop= True)
# test = test.reset_index(drop= True)
test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)


In [ ]:
# 加载 history 
t1 = pd.read_pickle(f'{newfeature_path}/history_feature_test2.pkl')
t1 = t1.reset_index(drop=True)

test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
t1 = pd.read_pickle(f'{newfeature_path}/history_feature1_test2.pkl')
t1 = t1.reset_index(drop=True)

test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
t1 = pd.read_pickle(f'{newfeature_path}/history_feature2_test2.pkl')
t1 = t1.reset_index(drop=True)

test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
# 加载 ans kfold feature
all_col = ['day', 'hour', 'q_inv_kfold_mean', 'q_inv_kfold_sum', 'q_inv_kfold_std', 'q_inv_kfold_count', 
           'u_inv_kfold_mean', 'u_inv_kfold_sum', 'u_inv_kfold_std', 'u_inv_kfold_count', 'q_ans_kfold_count',
           'u_ans_kfold_count', 'q_is_good_sum', 'q_is_good_max', 'q_is_good_mean', 'u_is_good_sum',
           'u_is_good_max', 'u_is_good_mean', 'q_is_rec_sum', 'q_is_rec_max', 'q_is_rec_mean', 'u_is_rec_sum',
           'u_is_rec_max', 'u_is_rec_mean', 'q_is_dest_sum', 'q_is_dest_max', 'q_is_dest_mean', 
           'u_is_dest_sum', 'u_is_dest_max', 'u_is_dest_mean', 'q_has_img_sum', 'q_has_img_max', 
           'q_has_img_mean', 'u_has_img_sum', 'u_has_img_max', 'u_has_img_mean', 'q_has_video_sum', 
           'q_has_video_max', 'q_has_video_mean', 'u_has_video_sum', 'u_has_video_max', 'u_has_video_mean',
           'q_word_count_sum', 'q_word_count_max', 'q_word_count_mean', 'u_word_count_sum', 'u_word_count_max',
           'u_word_count_mean', 'q_reci_cheer_sum', 'q_reci_cheer_max', 'q_reci_cheer_mean', 'u_reci_cheer_sum',
           'u_reci_cheer_max', 'u_reci_cheer_mean', 'q_reci_uncheer_sum', 'q_reci_uncheer_max', 
           'q_reci_uncheer_mean', 'u_reci_uncheer_sum', 'u_reci_uncheer_max', 'u_reci_uncheer_mean', 
           'q_reci_comment_sum', 'q_reci_comment_max', 'q_reci_comment_mean', 'u_reci_comment_sum', 
           'u_reci_comment_max', 'u_reci_comment_mean', 'q_reci_mark_sum', 'q_reci_mark_max', 
           'q_reci_mark_mean', 'u_reci_mark_sum', 'u_reci_mark_max', 'u_reci_mark_mean', 'q_reci_tks_sum',
           'q_reci_tks_max', 'q_reci_tks_mean', 'u_reci_tks_sum', 'u_reci_tks_max', 'u_reci_tks_mean',
           'q_reci_xxx_sum', 'q_reci_xxx_max', 'q_reci_xxx_mean', 'u_reci_xxx_sum', 'u_reci_xxx_max', 
           'u_reci_xxx_mean', 'q_reci_no_help_sum', 'q_reci_no_help_max', 'q_reci_no_help_mean', 
           'u_reci_no_help_sum', 'u_reci_no_help_max', 'u_reci_no_help_mean', 'q_reci_dis_sum', 
           'q_reci_dis_max', 'q_reci_dis_mean', 'u_reci_dis_sum', 'u_reci_dis_max', 'u_reci_dis_mean', 
           'q_diff_qa_days_sum', 'q_diff_qa_days_max', 'q_diff_qa_days_mean', 'u_diff_qa_days_sum', 
           'u_diff_qa_days_max', 'u_diff_qa_days_mean']
drop_col = ['u_is_rec_mean', 'u_reci_uncheer_mean', 'q_is_dest_sum', 'u_reci_uncheer_sum', 'u_is_rec_max', 
             'u_is_dest_mean','q_reci_uncheer_mean', 'q_reci_uncheer_sum', 'u_is_dest_sum', 'q_is_dest_max',
             'q_reci_uncheer_max', 'u_reci_tks_max', 'q_reci_mark_max','u_reci_dis_max', 'q_has_video_mean',
             'q_reci_no_help_mean', 'count_u_topic', 'u_has_video_mean', 'q_reci_dis_sum', 'q_reci_mark_sum',
             'q_reci_tks_sum','q_reci_tks_max','q_reci_dis_max','u_reci_mark_max','q_is_good_mean',
             'q_reci_no_help_sum', 'q_reci_xxx_max', 'u_reci_xxx_max','u_reci_no_help_sum','u_reci_xxx_sum',
              'u_is_good_mean','q_reci_no_help_max','u_has_img_max','u_is_good_sum','u_reci_no_help_max',
              'u_has_video_sum','uf_b5','q_reci_xxx_sum','q_is_good_sum','q_has_img_max','q_has_video_sum',
              'q_has_video_max','u_has_video_max','q_is_good_max','q_is_rec_max','u_is_good_max',
              'q_is_dest_mean','u_reci_uncheer_max','uf_c5_count','u_is_dest_max','q_is_rec_mean',
              'q_is_rec_sum','u_is_rec_sum', 'q_reci_xxx_mean','u_reci_xxx_mean','u_reci_comment_max',
              'q_reci_comment_sum','u_reci_cheer_max','u_reci_dis_sum','u_reci_tks_sum','q_has_img_sum',
              'q_reci_comment_max','q_reci_cheer_max','u_reci_no_help_mean','u_has_img_sum','u_reci_mark_sum']
use_col = list(set(all_col) - set(drop_col))


t1 = pd.read_csv(f'{newfeature_path}/test2_kfold_feature.txt', sep='\t', usecols=use_col)
test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
# test 重命名
t2 = []
n_dup = 0
for c in test.columns:
    if c not in t2:
        t2.append(c)
    else:
        t2.append(c+'_2')
        print(c, c+'_2')
        n_dup += 1
print('dup num: ', n_dup)

print(len(t2))
test.columns = t2
del test['day_2']
gc.collect()

In [ ]:
test['week'] = test['day']%7

In [ ]:
# 加载 invete feature 1
t1 = pd.read_csv(f'{newfeature_path}/test2_invite_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
# 加载 invete feature 2
t1 = pd.read_csv(f'{newfeature_path}/test2_invite_feature_2.txt', sep='\t')
test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
# 加载 kfold topic feature, QU
t1 = pd.read_csv(f'{newfeature_path}/newtest_kfold_topic_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
# 加载 user kfold topic feature，UU
t1 = pd.read_csv(f'{newfeature_path}/newtest_kfold_ut_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
t = test.dtypes
for x in t[t == 'int64'].index:
    test[x] = test[x].astype('int32')
for x in t[t == 'float64'].index:
    test[x] = test[x].astype('float32')
    
pickle.dump(test, open(f'{feature_path}/newtest_372.pkl', 'wb'))

In [ ]:
# test = pickle.load(open(f'{feature_path}/newtest_372.pkl', 'rb'))

In [ ]:
# 用户问题等信息 encoding
t1 = pd.read_pickle(f'{newfeature_path}/test_member_basic_feature.pkl')
test = pd.concat([test, t1], axis=1)
logging.info('test shape: %s', test.shape)

In [ ]:
# 计数特征
t1 = pd.read_pickle(f'{newfeature_path}/count_features.pkl')[9489162:].reset_index(drop=True)
logging.info('t1 shape: %s', t1.shape)


test = pd.concat([test, t1], axis=1)
logging.info('test shape: %s', test.shape)

In [ ]:
drop_feat = ['label', 'uid', 'qid', 'dt']
drop_feat += ['gender_count','qid_enc_day_diff_iq_day_mean','qid_enc_day_diff_iq_day_median','uid_enc_day_score_mean',
              'uid_enc_day_score_median','uf_b1_day_diff_iq_hour_mean','uf_b2_day_diff_iq_hour_mean','uf_b3_day_diff_iq_hour_mean',
              'qid_enc_day_hour_diff_iq_day_mean','qid_enc_day_hour_diff_iq_day_median','qid_enc_day_hour_diff_iq_hour_mean',
              'qid_enc_day_hour_diff_iq_hour_median','uid_enc_day_hour_score_mean','uid_enc_day_hour_score_median',
              'uid_enc_qid_enc_score_mean','uid_enc_qid_enc_score_median','uid_enc_wk_score_mean','uid_enc_wk_score_median',
              'freq_wk_score_median','freq_wk_diff_iq_hour_mean','gender_wk_diff_iq_day_median','gender_wk_diff_iq_hour_mean',
              'uf_b1_wk_diff_iq_day_median','uf_b1_wk_diff_iq_hour_mean','uf_b2_wk_score_median','uf_b2_wk_diff_iq_day_median',
              'uf_b2_wk_diff_iq_hour_mean','uf_b3_wk_diff_iq_hour_mean','uf_b4_wk_score_median','uf_b4_wk_diff_iq_day_median',
              'uf_b4_wk_diff_iq_hour_mean','uf_b5_wk_diff_iq_day_median','uf_b5_wk_diff_iq_hour_mean','uf_c5_wk_diff_iq_hour_mean']
drop_feat += ['uf_b2_max','uf_c5_max','dayuf_b2_max','qid_wk_uf_b3_median','qid_wk_uf_b1_max','dayuf_b1_max','qid_wk_uf_b2_median',
              'dayuf_b1_min','dayuf_c5_min','uf_b2_median','qid_wk_uf_b4_min','dayuf_b4_min','uf_b5_min','uf_c5_min','qid_wk_uf_b1_min',
              'qid_wk_uf_b5_max','uf_b1_max','dayuf_b3_max','dayuf_b2_min','uf_b4_max','uf_b3','uf_b5_max','qid_wk_uf_c5_median','uf_b4_min',
              'uf_b2_min','dayuf_b5_min','qid_wk_uf_b2_min','qid_wk_uf_b4_median','qid_wk_uf_b3_max','qid_enc_day_score_mean',
              'qid_wk_uf_c5_min','dayuf_b3_median','uf_b5_median','uf_b3_min','dayuf_b5_max','qid_wk_uf_b5_min','qid_wk_uf_b2_max',
              'dayuf_b3_min','dayuf_b4_max','qid_enc_day_score_median','qid_wk_uf_b1_median','dayuf_b2_median','dayuf_c5_max',
              'dayuf_b5_median','dayuf_c5_median']

cate_feats = ['qid_enc', 'uid_enc', 'freq', 'gender', 'uf_b1', 'uf_b2', 'uf_b3', 'uf_b4', 'uf_b5', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
val_feats = ['score', 'diff_iq_day', 'diff_iq_hour']
for fi in cate_feats:
    for fj in val_feats:
        sub_feats = ['mean', 'median']
        for sub in sub_feats:
            if fi+'_day_hour_'+fj+'_'+sub not in drop_feat:
                drop_feat.append(fi+'_day_hour_'+fj+'_'+sub)

last_100_feat = pickle.load(open('./last100col.pkl', 'rb'))
print(len(last_100_feat))
for f in last_100_feat:
    if f not in drop_feat:
        drop_feat.append(f)
            
feature_with_day = [x for x in test.columns if x not in drop_feat]
feature_cols = [x for x in test.columns if x not in drop_feat+['day']]
print(len(feature_with_day))

In [ ]:
col_854 = pickle.load(open('./0.854cols.pkl', 'rb'))
not_his_col = ['invite_hour', 'invite_day', 'ffa', 'sex', 'invite_day_count_y_median_x', 'invite_day_count_x_count_y', 
               'invite_day_count_y_std_y', 'invite_hour_std_y', 'invite_day_count_x_mean_x', 'invite_create', 'invite_day_count_x_mean_y', 
               'invite_hour_mean_y', 'invite_day_count_x_count_x', 'fa', 'salt_value', 'ffe', 'cross_topic', 'invite_day_count_y_count_x', 
               'sex_count', 'fd', 'invite_hour_std_x', 'invite_day_count_y', 'invite_day_count_y_median_y', 'create_hour', 
               'invite_hour_mean_x', 'fb', 'invite_day_count_x_median_y', 'fe', 'ffd', 'invite_day_count_x_std_x', 'ffc',
               'invite_day_count_y_mean_x', 'invite_day_count_x_std_y', 'invite_day_count_y_std_x', 'ffb', 'invite_day_count_x_median_x', 
               'invite_day_count_y_count_y', 'invite_day_count_x', 'create_day', 'fc', 'invite_day_count_y_mean_y']
for c1 in col_854:
    assert isinstance(c1, str)
    if (c1 not in feature_with_day) and (c1 not in not_his_col): 
        feature_with_day.append(c1)
        feature_cols.append(c1)
print(len(feature_with_day))
print(len(feature_cols))
print(len(set(feature_with_day)))

In [ ]:
# pickle.dump(feature_with_day, open('./feature_with_day.pkl', 'wb'))
# pickle.dump(feature_cols, open('./feature_cols.pkl', 'wb'))

In [ ]:
train_feature_with_day = pickle.load(open('./feature_with_day_in_train.pkl', 'rb'))
train_feature_cols = pickle.load(open('./feature_cols_in_train.pkl', 'rb'))

In [ ]:
len(train_feature_with_day)

In [ ]:
len(feature_with_day)

In [ ]:
t1 = pickle.load(open(f'{newfeature_path}/history_lastweek_test2_sup_a.pkl', 'rb')).reset_index(drop=True)
logging.info('t1 shape: %s', t1.shape)
test = pd.concat([test, t1], axis=1)
logging.info('test shape: %s', test.shape)

In [ ]:
t1 = pickle.load(open(f'{newfeature_path}/history_ltd6_test2_sup_a.pkl', 'rb')).reset_index(drop=True)
logging.info('t1 shape: %s', t1.shape)
test = pd.concat([test, t1], axis=1)
logging.info('test shape: %s', test.shape)

In [ ]:
t1 = pickle.load(open(f'{newfeature_path}/history_ltd_test2_sup_a.pkl', 'rb')).reset_index(drop=True)
logging.info('t1 shape: %s', t1.shape)
test = pd.concat([test, t1], axis=1)
logging.info('test shape: %s', test.shape)

In [ ]:
X_test = test[feature_cols]
X_test

In [ ]:
sub = pd.read_csv(f'{base_path}/invite_info_evaluate_2_0926.txt', sep='\t', header=None)
sub.columns = ['qid', 'uid', 'dt']
sub_size = len(sub)

logging.info("sub %s", sub.shape)

In [ ]:
assert len(X_test)==sub_size

In [ ]:
model_lgb = pickle.load(open('./model/lgb_label_3000_round.pkl', 'rb'))

In [ ]:
sub['label'] = model_lgb.predict(X_test[train_feature_cols])
# sub.to_csv('./result/1000_add_label_880059.txt', index=None, header=None, sep='\t')

In [ ]:
sub

In [ ]:
sub['label'].mean()

In [ ]:
sub2 = sub

In [ ]:
sub.to_csv('./result/3000_890439.txt', index=None, header=None, sep='\t')